# LROSE Wind Tutorial

---

This interactive tutorial takes you through the steps of how to run FRACTL and SAMURAI. FRACTL is a fast traditional solver with integrated interpolation using LROSE infrastructure, and it is able to perform both gridding and multi-Doppler synthesis for airborne radars and multiple ground-based radars. This is different than Radx2Grid which is only capable of gridding data for a single ground-based radar. FRACTL adopts techniques from older REORDER and CEDRIC software, but with some new improvements. In contrast to the older tools, in which interpolation is followed by synthesis, FRACTL does both steps together. FRACTL doesn’t require the CfRadial file with an aggregation of the sweeps, which is different than Radx2Grid. FRACTL performs a nearest neighbor interpolation of the reflectivity instead of bilinear interpolation in Radx2Grid. It combines the Doppler velocities together using the traditional 'normal' equations but with singular value decomposition of raw velocities in spherical space rather than Cramer's rule in gridded space like CEDRIC. 

SAMURAI is a variational analysis technique that is described in Bell et al. (2012), Foerster et al. (2014), Foerster and Bell (2017), Cha and Bell (2021), and other publications. The SAMURAI analysis yields a maximum likelihood estimate of the atmospheric state for a given set of observations and error estimates by minimizing a variational cost function. It has more features and more development than FRACTL, and is generally recommended over the former for publication-quality analysis. The two programs together provide a powerful combination to produce high quality multi-Doppler wind fields from ground-based, airborne, or mixed configurations.

---


## Tutorial Overview

### 1. Setup 

#### Directory organization 

<div>
<img src="../images/wind_structure.png" width="500"/>
</div>

#### QC-ed input data, a parameter file, and a center file:

**a. QC-ed radar data files will be provided in this example:**
* cfrad.20080702_230459.000_to_20080702_230948.000_CSU-PAWN_PPI.nc
* cfrad.20080702_230527.000_to_20080702_230946.000_CSU-CHIL_PPI.nc

*The QC process is not included in this tutorial. An example of one type of QC can be found in the QC tutorial and HawkEdit/solo are another tool to use for QC-ing data.

**b. Center file:**

*Note: The .cen file was generated by the provided Perl script. The center file is used in SAMURAI to define a static or moving Cartesian frame of reference for the analysis. It can be used to time-space correct the position of radar data in a moving storm system. It also sets the temporal limits for data to be included in the analysis. 
* Generate_center.pl
* 20080702.cen


**c. Parameter files:**
* fractl_params
* samurai_params

*Note: The parameter file have already been modified to run straight out of the box.*

#### Environment and packages¶

First, we import the required python packages to run this notebook. Most of the LROSE processing can be done with the os package and shell commands. 

In [ ]:
import os
#### Need to modify later
os.environ['BASE_DIR'] = '/home/jovyan/lrose-hub'
os.environ['RAW_DIR'] = '/home/jovyan/share/raw'
base_dir = os.environ['BASE_DIR']
!echo "Base directory: "$BASE_DIR

We will copy the required data to one directory as SAMURAI requires and organize the directory by performing the following commands:

In [ ]:
## make subdirectory within data for input and output directory
!rm -rf ${BASE_DIR}/data/wind/input
!mkdir -p ${BASE_DIR}/data/wind/input

# create samurai output directory
!rm -rf ${BASE_DIR}/data/wind/output_sam
!mkdir ${BASE_DIR}/data/wind/output_sam

# create samurai output directory
!rm -rf ${BASE_DIR}/data/wind/output_frac
!mkdir ${BASE_DIR}/data/wind/output_frac

# copy raw files to input directory
!cp ${RAW_DIR}/wind/*.nc ${BASE_DIR}/data/wind/input
!cp ${RAW_DIR}/wind/20080702.cen ${BASE_DIR}/data/wind/input


### 2. Prepare data for analysis

In this tutorial, the provided data have already been QC-ed. For a standard procedure, users are recommended to perform the following procedure:

* Convert radar data from level 2 or other native format to CfRadial using RadxConvert 
    - SAMURAI can also incorporate non-radar data, and can read many commonly used formats. All data files in the <code lang="bash">input</code> directory will be included in the analysis. The read subroutine for each datatype is determined by the file suffix.

* Quality-controlled raw data by desired QC-tools. See the QC tutorial for an example of how QC can be done; HawkEdit and Solo can also be used to QC data.

### 3. Set the FRACTL parameters

* If starting from scratch, generate a parameter file using
<code lang="bash">fractl -print_params > fractl_params</code>

* Modify the parameter file
    - Most of the defaults are fine for many analyses. The following are the minimum ones you need to set:
---
**CARTESIAN GRID**
| Parameter | Line # | Description |
| --- | --- | --- |
|zGrid, yGrid, xGrid| 174, 182, 190 |These determine the grid spacing and size of the domain. Provide either the grid spacing to determine the max and min from the data or provide "min,max,incr" for each parameter.|
|projLat0, projLon0| 229, 237 |The origin is an arbitrarily chosen point, but should be relevant for your objective. For example, it can be the geographical center of your multi-radar domain, the physical location of a radar for a single-radar domain, or the location of a feature of interest within your dataset (such as storm center). The latitude and longitude of your chosen origin should be given in decimal degrees.|
|radarAlt| 245 |Altitude of the radar (km). Sometimes mobile radars have the altitude entered in m, which can often cause errors in FRACTL.|


**FILES AND DIRECTORIES**
| Parameter | Line # | Description |
| --- | --- | --- |
|inDir| 391 |The input data directory. It can be a relative path, but an absolute path is more certain.|
|fileRegex| 399 |A regular expression to identify the files used for analysis. Usually ^cfrad, to find CfRadial files.|
|outNc| 426 |The output directory where the analysis will go.|
|outTxt| 417 |the name of the text file that will contain the verification of grid results.|


**FIELDS**
| Parameter | Line # | Description |
| --- | --- | --- |
|radialName| 440 |Variable name for the Doppler velocity.|
|dbzName| 448 |Variable name for the radar reflectivity.|
|ncpName| 456 |Variable name for the normalized coherent power. This variable can be used to do some simple QC thresholding. If you don't have NCP or SQI in your data then just point this to any other variable and select a value above which all data will be included.|
|minDbz| 58 |Any values below the minimum reflectivity value will be tossed out. Example: -20.|
|minNcp| 69 |Any values below the minimum NCP will be tossed out. Some radars do not have NCP, so the NCP variable can be replaced by any other variable to perform a simple quality control (e.g., SW < -1, dBZ < -20). This is generally intended for quick realtime quality control process.|

### 4. Run FRACTL

After modifying the parameter file, directing to the parameter file by typing the *-params* flags

In [ ]:
!fractl -params ${BASE_DIR}/params/wind/fractl_params 


### 5. Plot FRACTL results

#### Environment and packages
Import the packages to plot the FRACTL output. These are pre-installed for this tutorial, but if you run this notebook on your own computer you may need to install them yourself using the following code (where scipy is a representative example):

<code lang="bash">import sys</code>

<code lang="bash">!{sys.executable} -m pip install scipy</code> 

or 

<code lang="bash">!{sys.executable} -m conda install scipy</code>

depending on your system

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import xarray as xr
import matplotlib as mpl
import cartopy.crs as ccrs
from metpy.plots import ctables
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
mpl.rcParams['figure.dpi'] = 300


Load the netcdf file

In [ ]:
inDir = base_dir+"/data/wind/output_frac/20080702/"
file = "ncf_20080702_230948.nc"
ds_radar = xr.open_dataset(inDir+file).squeeze()
ds_radar.load()


In [ ]:
## Get variables:
da_DBZ = ds_radar['DBZ']
da_U = ds_radar['U']
da_V = ds_radar['V']
da_W = ds_radar['W']
da_CN = ds_radar['conditionNumber']

lon, lat = np.meshgrid( ds_radar.lon0, ds_radar.lat0 )


In [ ]:
## Set NWS reflectivity colorbar:
ref_norm, ref_cmap = ctables.registry.get_with_steps('NWSStormClearReflectivity', -20, 0.5)
plotting_alt=3*1000 # altitude at 3 km
plotting_var = ds_radar.DBZ.sel(z0=plotting_alt).data
plotting_var_u = ds_radar.U.sel(z0=plotting_alt).data
plotting_var_v = ds_radar.V.sel(z0=plotting_alt).data
plotting_lon = ds_radar.lon0
plotting_lat = ds_radar.lat0

fig = plt.figure(figsize=(12,12))
ax = plt.axes(projection=ccrs.PlateCarree())
# stamen_terrain = cimgt.Stamen('terrain')
# ax.add_image(stamen_terrain, 8)
    
ext = 0.02    
ax.set_extent([np.min(lon)+ext, np.max(lon)-ext, np.min(lat)+ext, np.max(lat)-ext], crs=ccrs.PlateCarree())
    
cf1 = ax.pcolormesh( plotting_lon, plotting_lat, plotting_var
                    , cmap=ref_cmap, norm=ref_norm
                    , alpha=0.8
                    , shading='auto'
                    , transform=ccrs.PlateCarree() 
                    )
step = 2
cf_q = ax.quiver( plotting_lon[::step, ::step], plotting_lat[::step, ::step]
                    , plotting_var_u[::step, ::step], plotting_var_v[::step, ::step]
                    , scale=200
                    , width=0.004
                    , color='k'
                    , transform=ccrs.PlateCarree() 
                    )

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

cbar_ax = fig.add_axes([0.95, 0.3, 0.02, 0.4])
cbar = fig.colorbar(cf1, cax=cbar_ax, fraction=0.04)
cbar.ax.tick_params(labelsize=14)
cbar.ax.set_title('[dBZ]', fontsize=14, y=-0.1)


### If you can see the above plot, then congratulations! You have successfully created and plotted a dual-Doppler analysis. Now let's try SAMURAI

### 6. Set the SAMURAI parameters

* One difference between FRACTL and SAMURAI is the requirement of a centerfile, which allows for a moving domain. Modify and run the Generate_center.pl perl script to generate a time-series of center locations by providing an estimation of the moving speed of the target and the reference location. The .cen file can be generated by typing the following command into a terminal:
<code lang="bash">./Generate_center.pl</code>

* Modify the parameter file

Like FRACTL, most of the defaults are fine for general usage. Here are the minimum parameters that need to be set:
---
**OPERATION SECTION**
| Parameter | Line # | Description |
| --- | --- | --- |
|data_directory| 171 |The input data directory. It can be a relative path, but an absolute path is more certain.|
|output_directory| 181 |The output directory where the analysis will go.|

**GRID DEFINITION SECTION**
| Parameter | Line # | Description |
| --- | --- | --- |
|i_min, i_max, i_incr| 254, 260, 266 |Define the minimum and maximum extent and grid spacing in km of the first horizontal dimension (e.g., Cartesian 'x' dimension). For cylindrical analyses (RTZ mode), this is radius.|
|j_min, j_max, j_incr| 272, 278, 284 |Define the minimum and maximum extent and grid spacing of the second horizontal dimension (e.g., Cartesian 'y' dimension in km). For cylindrical analyses (RTZ mode), this is azimuth (degrees).|
|k_min, k_max, k_incr| 290, 296, 302 |Define the minimum and maximum extent and grid spacing in km of the vertical dimension (i.e., 'z' dimension). k_min should usually be 0.|

**BACKGROUND SECTION**
| Parameter | Line # | Description |
| --- | --- | --- |
|ref_time| 324 |This is the reference time to which all data will be time-space corrected. It must be a valid time in the centerfile or an error will occur. The time should be the same as the origin time corresponding to the latitude/longitude in the Perl center file script.|

**RADAR SECTION**
| Parameter | Line # | Description |
| --- | --- | --- |
|radar_skip| 352 |This option can be used to skip beams in the radar data. When set to '1', all beams are used. Setting it to 2 speeds up the calculation and is useful for preliminary analysis. It should generally be set to 1 for final analysis.|
|radar_stride| 358 |This option sets the number of gates over which averaging occurs along the beam. A stride of '1' uses all data, and higher numbers average multiple gates of the given stride. Higher numbers speed up the calculation and can be used to thin the data to achieve desired spatial resolution of the input velocities.|
|radar_dbz| 382 |The name of the reflectivity field in the radar data.|
|radar_vel| 392 |The name of the Doppler velocity field in the radar data.|
|radar_sw| 398 |The name of the spectrum width field in the radar data. This value is used to set the observational error for the Doppler velocity. If you don't have spectrum width, you can point it at another variable (like NCP) or create a simple uniform field.|
|i_reflectivity_roi, j_reflectivity_roi, k_reflectivity_roi| 404, 410, 416 |These variables set the radius of influence for the reflectivity interpolation. They should be similar to the grid increments defined above.|
|mask_reflectivity| 428 |The analysis can be set to missing data where there is no reflectivity. If set to 'None' then no masking is performed. A numerical value will be used as a threshold for the masking, with all data at nodes having less than the given reflectivity value removed. If you are not using a background field then it is generally a good idea to set this to some small number (like -10 dBZ) since the winds are not valid if there is no radar data.|

The following are optional parameters:
---
**BACKGROUND SECTION**
| Parameter | Line # | Description |
| --- | --- | --- |
|load_background| 93 |TRUE: A first guess of the analysis stored in the <code lang="bash">samurai_Background.in</code> will be loaded. This can come from a model, sounding, or other data source if available. Most of the time it is not available so it should be set to FALSE (default).|
|adjust_background| 114 |TRUE: the background will be adjusted to satisfy the mass continuity and match the supplied data using a variational adjustment. Without this, it is just interpolated simply to the grid.|

**OPERATION SECTION**
| Parameter | Line # | Description |
| --- | --- | --- |
|mode| 148 |The default mode is MODE_XYZ, which is Cartesian grid. You can run in cylindrical mode with MODE_RTZ. Stay tuned for more radar centric RTZ mode in the near future!|
|preprocess_obs| 189 |TRUE: the raw data files will be preprocessed according to their file suffixes, and a <code lang="bash">samurai_Observations.in</code> file will be generated. <br> FALSE: SAMURAI will load the observations from the <code lang="bash">samurai_Observations.in</code> file located in the data_directory. This is useful for generating fake data, or for saving some time if you already ran the preprocessing of lots of files.|
|num_iterations| 200 |SAMURAI has the ability to take the output analysis and use it as a first guess for another analysis. This can be useful to create a 'coarse' analysis, followed by a 'fine' analysis. It is very important to set the observation and background errors appropriately if you choose this option.|

**BACKGROUND SECTION**
| Parameter | Line # | Description |
| --- | --- | --- |
|ref_state| 314 |A sounding file to define the hydrostatic reference state used in the analysis. The default is the Dunion (2011) moist tropical sounding file. The analysis is generally not sensitive to this sounding, as it is just used to provide a reference density field and fall speed relationships in most cases. If you are analyzing thermodynamic information or have a very different freezing level it may be useful to provide a more appropriate sounding. The file format is similar to that of WRF or CM1 idealized soundings.|
|i_background_roi, j_background_roi| 332, 340 |These set the background radius of influence when loading a background field.|

**RADAR SECTION**
| Parameter | Line # | Description |
| --- | --- | --- |
|qr_variable| 372 |In the default 'dbz' mode, the reflectivity is just interpolated and not included in the cost function minimization. If this is set to 'qr', then reflectivity is converted to liquid water using Z-M relationships defined in Gamache et al. (1993) and used as an additional variable in the cost function minimization. This is useful if you have other liquid water measurements to assimilate. Since this is a relatively simple Z-M it should be used with caution.|
|dbz_pseudow_weight| 422 |SAMURAI has the ability to set a 'soft' w=0 boundary condition at echo top, and setting this weight will determine how soft or hard that constraint is enforced. Usually a hard w=0 is enforced via the spline boundary conditions at the bottom and top of the domain, so this is optional. It can be useful if the vertical velocities are believed to be too strong at the top of the cloud.|
|melting_zone_width, mixed_phase_dbz, rain_dbz| 434, 440, 446 |SAMURAI has some basic terminal fall speed corrections that use reflectivity and Z-VT relationships. These parameters control which relationships are used depending on dbz and the height of the zero C level (which is determined from the reference sounding).|

**BOUNDARY CONDITIONS SECTION**

Available options are R0, R1T0, R1T1, R1T2, R2T10, R2T20, R3, and PERIODIC following Ooyama (2002). The default "non"- boundary condition (R0) adds a buffer set of gridpoints that are used to calculate the solution but are discarded for output. Different boundary conditions can be set on the left (L) or right (R) side of the domain for each variable and dimension. The most common option other than R0 would be R1T0 for vertical velocity (rhow = 0) at the surface and/or domain top. Periodic domains are only valid for the i and j dimension, but are available in both the XYZ and RTZ mode. Improved boundary conditions are currently in development.

**OBSERVATION ERRORS SECTION**

Specified error is given in terms of a standard deviation, and is fixed for all observations from a particular instrument except radar. In the radar case, the spectrum width and elevation angle (proportional to terminal fall speed contribution) are used to define the error for each radar gate. A minimum error (radar_min_error) is also enforced.

**ITERATION DEPENDENT SECTION**
| Parameter | Line # | Description |
| --- | --- | --- |
|mc_weight| 1137 |Specify the weight given to the mass continuity constraint. Default is set to 1, and is generally not recommended to change unless you have a good reason.|
|i_filter_length, j_filter_length, k_filter_length| 1247, 1257, 1267 |Gaussian recursive filter is a low-pass filter. Smaller filter lengths retain more detail, and larger filter lengths smooth more. The general recommendation is 4, 4, 2, which removes features less than approximately 4dx, 4dy, and 2dz from the analysis.|
|i_spline_cutoff, j_spline_cutoff, k_spline_cutoff| 1277, 1287, 1297 |Spline cutoff is implemented as a third derivative constraint on the cubic B-spline basis during the spline transform. It is a sharper filter than the Gaussian, roughly equivalent to a sixth order filter.|
|i_max_wavenumber, j_max_wavenumber, k_max_wavenumber| 1307, 1317, 1327 |The Fourier spectral filter is the sharpest filter. It can be used for desired effects, such as explicitly removing high-wavenumber features in the spectral domain (usually for RTZ analysis), or for producing a 'mean' field by restricting to wavenumber zero.|

### 7. Run SAMURAI

After modifying the parameter file, directing to the parameter file by typing the *-params* flags

In [ ]:
!samurai -params ${BASE_DIR}/params/wind/samurai_params 


### 8. Plot SAMURAI results

#### Environment and packages¶
The packages were already imported when we plotted the FRACTL results.

#### Load the netcdf file

In [ ]:
inDir_s = base_dir+"/data/wind/output_sam/"
file_s = "samurai_XYZ_analysis.nc"
ds_radar_s = xr.open_dataset(inDir_s+file_s).squeeze()
ds_radar_s.load()


In [ ]:
## Get variables:
da_s_DBZ = ds_radar_s['DBZ']
da_s_U = ds_radar_s['U']
da_s_V = ds_radar_s['V']
da_s_W = ds_radar_s['W']

lon_s, lat_s = np.meshgrid( ds_radar_s.longitude, ds_radar_s.latitude )


In [ ]:
## Set NWS reflectivity colorbar:
ref_norm, ref_cmap = ctables.registry.get_with_steps('NWSStormClearReflectivity', -20, 0.5)
plotting_alt=3 # altitude at 3 km
plotting_var_s = ds_radar_s.DBZ.sel(altitude=plotting_alt)
plotting_var_u_s = ds_radar_s.U.sel(altitude=plotting_alt).data
plotting_var_v_s = ds_radar_s.V.sel(altitude=plotting_alt).data
plotting_lon_s = ds_radar_s.longitude
plotting_lat_s = ds_radar_s.latitude

fig = plt.figure(figsize=(12,12))
ax = plt.axes(projection=ccrs.PlateCarree())
# stamen_terrain = cimgt.Stamen('terrain')
# ax.add_image(stamen_terrain, 8)
    
ext = 0.02    
ax.set_extent([np.min(lon_s)+ext, np.max(lon_s)-ext, np.min(lat_s)+ext, np.max(lat_s)-ext], crs=ccrs.PlateCarree())
    
cf1 = ax.pcolormesh( plotting_lon_s, plotting_lat_s, plotting_var_s
                    , cmap=ref_cmap, norm=ref_norm
                    , alpha=0.8
                    , shading='auto'
                    , transform=ccrs.PlateCarree() 
                    )
step = 2
cf_q = ax.quiver( plotting_lon_s[::step], plotting_lat_s[::step]
                    , plotting_var_u_s[::step, ::step], plotting_var_v_s[::step, ::step]
                    , scale=200
                    , width=0.004
                    , color='k'
                    , transform=ccrs.PlateCarree() 
                    )

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

cbar_ax = fig.add_axes([0.95, 0.3, 0.02, 0.4])
cbar = fig.colorbar(cf1, cax=cbar_ax, fraction=0.04)
cbar.ax.tick_params(labelsize=14)
cbar.ax.set_title('[dBZ]', fontsize=14, y=-0.1)


### 9. Apply FRACTL Condition Number

The FRACTL 'Condition Number' represents how well-posed the geometry is for a particular multi-Doppler retrieval. It is similar to the USTD and VSTD field from CEDRIC, which represent the standard deviation of the expected error in the resolved wind field based on the geometry. Since SAMURAI solves for the wind field globally using spline basis functions, it is difficult to calculate this error at each gridpoint. If a FRACTL analysis is run on the same domain, then it can be used to remove regions with poor geometry. Here, we use a value of 10 to threshold the SAMURAI analysis.

In [ ]:
# Read file into radar object
inDir_f = base_dir+"/data/wind/output_frac/20080702/"
file_f = "ncf_20080702_230948.nc"
ds_radar_f = xr.open_dataset(inDir_f+file_f).squeeze()
ds_radar_f


In [ ]:
plotting_CN = ds_radar_f.conditionNumber.sel(z0=3.0*1000)


In [ ]:
CN_threshold = 10
dbz_goodcondition = np.where(plotting_CN < CN_threshold, plotting_var_s, np.nan)
u_goodcondition = np.where(plotting_CN < CN_threshold, plotting_var_u_s, np.nan)
v_goodcondition = np.where(plotting_CN < CN_threshold, plotting_var_v_s, np.nan)


In [ ]:
fig = plt.figure(figsize=(10,10))
ax = plt.axes(projection=ccrs.PlateCarree())
    
ext = 0.02    
ax.set_extent([np.min(lon_s)+ext, np.max(lon_s)-ext, np.min(lat_s)+ext, np.max(lat_s)-ext], crs=ccrs.PlateCarree())
    
cf1 = ax.pcolormesh( plotting_lon_s, plotting_lat_s, dbz_goodcondition
                    , cmap=ref_cmap, norm=ref_norm
                    , alpha=0.8
                    , shading='auto'
                    , transform=ccrs.PlateCarree() 
                    )
step = 2
cf_q = ax.quiver( plotting_lon_s[::step], plotting_lat_s[::step]
                    , u_goodcondition[::step, ::step], v_goodcondition[::step, ::step]
                    , scale=200
                    , width=0.004
                    , color='k'
                    , transform=ccrs.PlateCarree() 
                    )

gl = ax.gridlines(crs=ccrs.PlateCarree(), draw_labels=True,
                  linewidth=2, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False
gl.xformatter = LONGITUDE_FORMATTER
gl.yformatter = LATITUDE_FORMATTER

cbar_ax = fig.add_axes([0.95, 0.3, 0.02, 0.4])
cbar = fig.colorbar(cf1, cax=cbar_ax, fraction=0.04)
cbar.ax.tick_params(labelsize=14)
cbar.ax.set_title('[dBZ]', fontsize=14, y=-0.1)


### Congratulations! You have successfully completed the LROSE Wind tutorial. There are other options to set in both FRACTL and SAMURAI, but the steps and parameters in this tutorial can produce a good quality wind field in many cases. Be critical with your own analysis and feel free to reach out to the LROSE team for questions as you analyze your own data.